<a href="https://colab.research.google.com/github/szeshiang/Deep-Learning/blob/main/hyperparameter_tuning_2_Mar_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

why need hyper-parameter tuning?
- to find the optimiser that gives the best model performance

In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.4 MB/s eta 0:00:00


In [2]:
from tensorflow.keras.layers import Dense

In [4]:
import keras_tuner

In [5]:
from keras_tuner.tuners import RandomSearch

In [6]:
# import dataset
from tensorflow.keras.datasets import mnist
print('Data loaded')

Data loaded


In [7]:
(X_train,y_train),(X_test,y_test)=mnist.load_data()
X_train=X_train/255.0
X_test=X_test/255.0

11490434/11490434 [==============================] - 0s 0us/step


In [8]:
X_train.shape

(60000, 28, 28)

In [9]:
X_test.shape

(10000, 28, 28)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras.optimizers import Adam

In [23]:
def build_model(hp):  # build_model defines the model architecture
  model=Sequential()
  model.add(Flatten(input_shape=(28,28)))
  for i in range(hp.Int('num_layers',2,20)):  # search for num of layers between 2 to 20
      model.add(Dense(units=hp.Int('units_'+ str(i),min_value=32,max_value=512,step=32), activation='relu'))
      model.add(Dropout(hp.Float('rate'+ str(i), min_value=0.0, max_value=0.5, step=0.1)))
  model.add(Dense(10,activation='softmax'))
  model.compile(optimizer=Adam(hp.Choice('learning_rate',[1e-2,1e-3,1e-4])), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

In [44]:
# tuner
tuner=RandomSearch(build_model,
                   objective='val_accuracy',
                   max_trials=2,  # tuner will explore 2 diff sets of hyperparameters
                   max_retries_per_trial=1,
                   directory='my_dir',
                   project_name='hyperp_tuner')

Reloading Tuner from my_dir/hyperp_tuner/tuner0.json


In [45]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [46]:
tuner.results_summary()

Results summary
Results in my_dir/hyperp_tuner
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layers: 19
units_0: 192
rate0: 0.30000000000000004
units_1: 32
rate1: 0.30000000000000004
learning_rate: 0.001
units_2: 32
rate2: 0.0
units_3: 32
rate3: 0.0
units_4: 32
rate4: 0.0
units_5: 32
rate5: 0.0
units_6: 32
rate6: 0.0
units_7: 32
rate7: 0.0
units_8: 32
rate8: 0.0
units_9: 32
rate9: 0.0
units_10: 32
rate10: 0.0
units_11: 32
rate11: 0.0
units_12: 32
rate12: 0.0
units_13: 32
rate13: 0.0
units_14: 32
rate14: 0.0
units_15: 32
rate15: 0.0
units_16: 32
rate16: 0.0
units_17: 32
rate17: 0.0
units_18: 32
rate18: 0.0
Score: 0.9546999931335449

Trial 1 summary
Hyperparameters:
num_layers: 8
units_0: 288
rate0: 0.2
units_1: 96
rate1: 0.0
learning_rate: 0.01
units_2: 320
rate2: 0.1
units_3: 512
rate3: 0.2
units_4: 128
rate4: 0.1
units_5: 512
rate5: 0.1
units_6: 288
rate6: 0.1
units_7: 224
rate7: 0.0
units_8: 96
rate8: 0.1
units_9: 416
rat

In [47]:
best_hps=tuner.get_best_hyperparameters()[0]
print(best_hps)

In [48]:
model=tuner.hypermodel.build(best_hps)
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 dense_69 (Dense)            (None, 192)               150720    
                                                                 
 dropout_65 (Dropout)        (None, 192)               0         
                                                                 
 dense_70 (Dense)            (None, 32)                6176      
                                                                 
 dropout_66 (Dropout)        (None, 32)                0         
                                                                 
 dense_71 (Dense)            (None, 32)                1056      
                                                                 
 dropout_67 (Dropout)        (None, 32)               

In [49]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
1875/1875 [==============================] - 16s 7ms/step - loss: 1.0329 - accuracy: 0.6207 - val_loss: 0.3804 - val_accuracy: 0.9025
Epoch 2/5
1875/1875 [==============================] - 12s 7ms/step - loss: 0.4283 - accuracy: 0.8810 - val_loss: 0.2732 - val_accuracy: 0.9326
Epoch 3/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.3589 - accuracy: 0.9031 - val_loss: 0.2504 - val_accuracy: 0.9387
Epoch 4/5
1875/1875 [==============================] - 12s 7ms/step - loss: 0.3326 - accuracy: 0.9110 - val_loss: 0.2226 - val_accuracy: 0.9461
Epoch 5/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.3021 - accuracy: 0.9208 - val_loss: 0.2901 - val_accuracy: 0.9251
